In [45]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import brier_score_loss

folder = 'march-machine-learning-mania-2024/'

Seeds = pd.read_csv(folder+'WNCAATourneySeeds.csv')
Tourney_Compact = pd.read_csv(folder+'WNCAATourneyCompactResults.csv')

SeedDictionary = Seeds.set_index(['Season', 'TeamID'])

TourneyInput = pd.DataFrame()

winIds = Tourney_Compact['WTeamID']
loseIds = Tourney_Compact['LTeamID']
season = Tourney_Compact['Season']

winners = pd.DataFrame()
winners[['Season', 'Team1', 'Team2']] = Tourney_Compact[['Season', 'WTeamID', 'LTeamID']]
winners['Result'] = 1

losers = pd.DataFrame()
losers[['Season', 'Team1', 'Team2']] = Tourney_Compact[['Season', 'LTeamID', 'WTeamID']]
losers['Result'] = 0

TourneyInput = pd.concat([winners, losers])

# Filter TourneyInput to include only seasons from 2015 onwards
TourneyInput = TourneyInput[TourneyInput['Season'] >= 2015]

Team1seeds = []
Team2seeds = []

for x in range(len(TourneyInput)):
    try:
        index = (TourneyInput.iloc[x]['Season'], TourneyInput.iloc[x]['Team1'])
        seed = SeedDictionary.loc[index].values[0]
        if len(seed) == 4:
            seed = int(seed[1:-1])
        else:
            seed = int(seed[1:])
        Team1seeds.append(seed)
    except KeyError:
        Team1seeds.append(0)

    try:
        index = (TourneyInput.iloc[x]['Season'], TourneyInput.iloc[x]['Team2'])
        seed = SeedDictionary.loc[index].values[0]
        if len(seed) == 4:
            seed = int(seed[1:-1])
        else:
            seed = int(seed[1:])
        Team2seeds.append(seed)
    except KeyError:
        Team2seeds.append(0)

TourneyInput['Team1Seed'] = Team1seeds
TourneyInput['Team2Seed'] = Team2seeds

TourneyInput['SeedDiff'] = TourneyInput['Team2Seed'] - TourneyInput['Team1Seed']

stats = TourneyInput[['SeedDiff']].values
results = TourneyInput['Result'].values

x = stats
y = results
np.random.seed(1)
index = np.random.permutation(len(x))
TrainIndex = index[:int(-.2*len(x))]
TestIndex = index[int(-.2*len(x)):]

XTrain = x[TrainIndex]
XTest = x[TestIndex]
YTrain = y[TrainIndex]
YTest = y[TestIndex]

model = RandomForestClassifier(random_state=1)
model = model.fit(XTrain, YTrain)

# Predict probabilities for the test set
y_pred_prob = model.predict_proba(XTest)[:, 1]

# Calculate the Brier Score
brier_score = brier_score_loss(YTest, y_pred_prob)

print("Brier Score:", brier_score)

# Load the sample dataframe
womens_df = pd.read_csv('2024_potential_matchups.csv')

# Prepare the input features for prediction
Team1seeds = []
Team2seeds = []

for x in range(len(womens_df)):
    try:
        index = (2023, womens_df.iloc[x]['T1_ID'])
        seed = SeedDictionary.loc[index].values[0]
        if len(seed) == 4:
            seed = int(seed[1:-1])
        else:
            seed = int(seed[1:])
        Team1seeds.append(seed)
    except KeyError:
        Team1seeds.append(0)

    try:
        index = (2023, womens_df.iloc[x]['T2_ID'])
        seed = SeedDictionary.loc[index].values[0]
        if len(seed) == 4:
            seed = int(seed[1:-1])
        else:
            seed = int(seed[1:])
        Team2seeds.append(seed)
    except KeyError:
        Team2seeds.append(0)

womens_df['Team1Seed'] = Team1seeds
womens_df['Team2Seed'] = Team2seeds
womens_df['SeedDiff'] = womens_df['Team2Seed'] - womens_df['Team1Seed']

stats = womens_df[['SeedDiff']].values

# Predict probabilities for the women's division
y_pred_prob = model.predict_proba(stats)[:, 1]

# Add the predicted probabilities to the sample dataframe
womens_df['T1_Win_Prob'] = y_pred_prob

#Round two decimal places
womens_df['T1_Win_Prob'] = womens_df['T1_Win_Prob'].round(2)

# Select the desired columns for the final DataFrame
womens_df = womens_df[['T1_Spelling', 'T2_Spelling', 'T1_Win_Prob']]

# Save the updated DataFrame to a new CSV file
womens_df.to_csv('2024_potential_matchups_with_predictions_womens.csv', index=False)

Brier Score: 0.1461868557806418
